# InceptionTime Model - TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from utils.metrics import calculate_metrics, print_metrics_summary
from utils.visualization import save_visualizations

In [ ]:
class InceptionTime(Model):
    def __init__(self, input_shape=(24, 4, 1), num_classes=8, model_dir="saved_models"):
        super(InceptionTime, self).__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model_dir = model_dir
        os.makedirs(self.model_dir, exist_ok=True)

        # CNN Pathway
        self.conv1 = layers.Conv2D(32, (3,3), activation='relu', padding='same')
        self.pool1 = layers.MaxPooling2D(pool_size=(2,2))
        self.conv2 = layers.Conv2D(64, (3,3), activation='relu', padding='same')
        self.pool2 = layers.MaxPooling2D(pool_size=(2,2))
        self.conv3 = layers.Conv2D(128, (3,3), activation='relu', padding='same')
        self.pool3 = layers.MaxPooling2D(pool_size=(2,2), padding='same')
        self.flatten_cnn = layers.Flatten()

        # TCN Pathway
        self.tcn1 = layers.Conv2D(64, (7,1), dilation_rate=2, activation='relu', padding='same')
        self.tcn2 = layers.Conv2D(64, (7,1), dilation_rate=4, activation='relu', padding='same')
        self.tcn3 = layers.Conv2D(64, (7,1), dilation_rate=8, activation='relu', padding='same')

        # Inception Modules
        self.conv1x1 = layers.Conv2D(64, (1,1), activation='relu', padding='same')
        self.conv3x3_d2 = layers.Conv2D(64, (3,3), dilation_rate=2, activation='relu', padding='same')
        self.conv5x5_d4 = layers.Conv2D(64, (5,5), dilation_rate=4, activation='relu', padding='same')
        self.concatenated = layers.Concatenate()

        # Residual Connection
        self.residual = layers.Add()
        self.flatten_tcn = layers.Flatten()

        # Output
        self.output_layer = layers.Dense(num_classes, activation='softmax')

        # Build the model
        self.build((None,) + input_shape)

In [ ]:
    def call(self, inputs):
        # CNN Pathway
        conv1_out = self.conv1(inputs)
        pool1_out = self.pool1(conv1_out)
        conv2_out = self.conv2(pool1_out)
        pool2_out = self.pool2(conv2_out)
        conv3_out = self.conv3(pool2_out)
        pool3_out = self.pool3(conv3_out)
        flatten_cnn_out = self.flatten_cnn(pool3_out)

        # TCN Pathway
        tcn1_out = self.tcn1(inputs)
        tcn2_out = self.tcn2(tcn1_out)
        tcn3_out = self.tcn3(tcn2_out)

        # Inception Module
        conv1x1_out = self.conv1x1(tcn3_out)
        conv3x3_d2_out = self.conv3x3_d2(tcn3_out)
        conv5x5_d4_out = self.conv5x5_d4(tcn3_out)
        concatenated_out = self.concatenated([conv1x1_out, conv3x3_d2_out, conv5x5_d4_out])

        # Residual Connection
        residual_out = self.residual([tcn2_out, tcn3_out])
        flatten_tcn_out = self.flatten_tcn(residual_out)

        # Combine Pathways
        concatenated_output = self.concatenated([flatten_cnn_out, flatten_tcn_out])
        return self.output_layer(concatenated_output)

In [ ]:
    def train(self, train_x, train_y, valid_x, valid_y, 
              batch_size=32, epochs=100, learning_rate=1e-4, 
              model_name="InceptionTime"):
        self.encoder = LabelEncoder()
        train_y_enc = self.encoder.fit_transform(train_y)
        valid_y_enc = self.encoder.transform(valid_y)

        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=os.path.join(self.model_dir, f"{model_name}_best.h5"),
                monitor='val_accuracy',
                save_best_only=True,
                save_weights_only=False,
                mode='max',
                verbose=1
            ),
            tf.keras.callbacks.EarlyStopping(
                patience=10,
                restore_best_weights=True
            )
        ]

        self.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )

        history = self.fit(
            x=train_x,
            y=train_y_enc,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(valid_x, valid_y_enc),
            callbacks=callbacks
        )

        self.save_model(model_name)
        return history

In [ ]:
    def save_model(self, model_name):
        save_path = os.path.join(self.model_dir, f"{model_name}_final.h5")
        self.save(save_path)
        print(f"Model saved to {save_path}")

    def load_model(self, model_path):
        self = tf.keras.models.load_model(model_path)
        print(f"Model loaded from {model_path}")
        return self

    def evaluate(self, test_x, test_y, model_name="InceptionTime", class_names=None):
        test_y_enc = self.encoder.transform(test_y)
        y_pred = np.argmax(self.predict(test_x), axis=1)

        metrics = calculate_metrics(test_y_enc, y_pred, model_name)
        print_metrics_summary(metrics)

        save_visualizations(
            model=self,
            x_data=test_x,
            y_true=test_y_enc,
            y_pred=y_pred,
            model_name=model_name,
            class_names=class_names
        )

        return metrics